In [3]:
#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [20]:
CLIENT_ID = '02XI22LSV2ONB4TDCQKNMUXPECHUXN0BHAFDKXUFRLPQSY1Q' # your Foursquare ID
CLIENT_SECRET = 'KZFNH24O4EIKJIN35YNZW5CXKMBSCLGIPI2KXQXF2Y2TEQGG' # your Foursquare Secret
VERSION = '20180604'
#LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 02XI22LSV2ONB4TDCQKNMUXPECHUXN0BHAFDKXUFRLPQSY1Q
CLIENT_SECRET:KZFNH24O4EIKJIN35YNZW5CXKMBSCLGIPI2KXQXF2Y2TEQGG


In [21]:
#search_query = 'Chinese'
#radius = 5000
near = "San Antonio, TX"
#print(search_query + ' .... OK!')

In [26]:
base_url = 'https://api.foursquare.com/v2/venues/search'
parameters = {"near": near ,  "client_id": CLIENT_ID , "client_secret" :  CLIENT_SECRET , "v" :VERSION, "limit": 1000}
#client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
base_url,parameters


('https://api.foursquare.com/v2/venues/search',
 {'near': 'San Antonio, TX',
  'client_id': '02XI22LSV2ONB4TDCQKNMUXPECHUXN0BHAFDKXUFRLPQSY1Q',
  'client_secret': 'KZFNH24O4EIKJIN35YNZW5CXKMBSCLGIPI2KXQXF2Y2TEQGG',
  'v': '20180604',
  'limit': 1000})

In [27]:
#results = requests.get(url).json()
results = requests.get(base_url, params = parameters).json()
#results

In [28]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ad4bffef964a520f3ea20e3,115 W Main Plz,...,NaN,"[115 W Main Plz, San Antonio, TX 78205, United...","[{'label': 'display', 'lat': 29.42445959712935...",29.424460,-98.493613,78205,TX,San Fernando Cathedral,v-1545154288,NaN
1,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bb08897f964a520464a3ce3,100 Dolorosa,...,NaN,"[100 Dolorosa, San Antonio, TX 78205, United S...","[{'label': 'display', 'lat': 29.42321234496511...",29.423212,-98.493719,78205,TX,Bexar County Courthouse,v-1545154288,NaN
2,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c814f67dc018cfabf7ec46c,300 Dolorosa,...,Dwyer,"[300 Dolorosa (Dwyer), San Antonio, TX 78205, ...","[{'label': 'display', 'lat': 29.4243144745312,...",29.424314,-98.493832,78205,TX,227th District Court,v-1545154288,NaN
3,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bbe1906b42aef3b97664bc3,100 W Main Plz,...,btwn E Commerce St & W Market/Dolorosa,[100 W Main Plz (btwn E Commerce St & W Market...,"[{'label': 'display', 'lat': 29.42458767477948...",29.424588,-98.493285,78205,TX,Main Plaza Park,v-1545154288,NaN
4,"[{'id': '4bf58dd8d48988d1f1931735', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,False,50b4feace4b0818e9afd0b96,NaN,...,NaN,"[San Antonio, TX 78205, United States]","[{'label': 'display', 'lat': 29.42475970408229...",29.424760,-98.493585,78205,TX,Main Plaza Farmer's Market,v-1545154288,NaN


In [58]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered

In [59]:
dataframe_filtered.shape

(197, 14)

In [205]:
print('{} venues were returned by Foursquare.'.format(dataframe_filtered.shape[0]))

197 venues were returned by Foursquare.


In [206]:
dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id,Cluster Labels
0,San Fernando Cathedral,Church,115 W Main Plz,US,San Antonio,United States,NaN,"[115 W Main Plz, San Antonio, TX 78205, United...","[{'label': 'display', 'lat': 29.42445959712935...",29.424460,-98.493613,78205,TX,4ad4bffef964a520f3ea20e3,1
1,Bexar County Courthouse,Courthouse,100 Dolorosa,US,San Antonio,United States,NaN,"[100 Dolorosa, San Antonio, TX 78205, United S...","[{'label': 'display', 'lat': 29.42321234496511...",29.423212,-98.493719,78205,TX,4bb08897f964a520464a3ce3,0
2,227th District Court,Courthouse,300 Dolorosa,US,San Antonio,United States,Dwyer,"[300 Dolorosa (Dwyer), San Antonio, TX 78205, ...","[{'label': 'display', 'lat': 29.4243144745312,...",29.424314,-98.493832,78205,TX,4c814f67dc018cfabf7ec46c,0
3,Main Plaza Park,Plaza,100 W Main Plz,US,San Antonio,United States,btwn E Commerce St & W Market/Dolorosa,[100 W Main Plz (btwn E Commerce St & W Market...,"[{'label': 'display', 'lat': 29.42458767477948...",29.424588,-98.493285,78205,TX,4bbe1906b42aef3b97664bc3,0
4,Main Plaza Farmer's Market,General Entertainment,NaN,US,San Antonio,United States,NaN,"[San Antonio, TX 78205, United States]","[{'label': 'display', 'lat': 29.42475970408229...",29.424760,-98.493585,78205,TX,50b4feace4b0818e9afd0b96,0


In [123]:
def getNearbyVenues(df, radius=500):
    
    venues_list=[]
    #for name, lat, lng in zip(names, latitudes, longitudes):
    for index, row in df.iterrows() :
        name = row['name']
        lat = row['lat']
        lng = row['lng']
        #print(row['name'])
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            1000)
            
        # make the GET request
        results = requests.get(url).json()["response"]
        
        if 'groups' in  results :
            results = results['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        else :
            venue_list.append([(name, lat, lng,"","","","")])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

In [124]:
nearby_val = getNearbyVenues(dataframe_filtered)


In [209]:
nearby_val.head()
#nearby_val.shape

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,San Fernando Cathedral,29.42446,-98.493613,San Fernando Cathedral,29.424460,-98.493613,Church
1,San Fernando Cathedral,29.42446,-98.493613,The Esquire Tavern,29.424834,-98.491761,Cocktail Bar
2,San Fernando Cathedral,29.42446,-98.493613,Biga on the Banks,29.423520,-98.492137,American Restaurant
3,San Fernando Cathedral,29.42446,-98.493613,Drury Plaza Hotel San Antonio Riverwalk,29.424357,-98.491746,Hotel
4,San Fernando Cathedral,29.42446,-98.493613,Main Plaza Park,29.424588,-98.493285,Plaza


In [125]:
nearby_val.groupby('Neighbourhood').count()
#nearby_val

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
0.0 BEGIN,100,100,100,100,100,100
131st District Court,100,100,100,100,100,100
144th District Court of Bexar County,100,100,100,100,100,100
150th District Court,100,100,100,100,100,100
166th District Court,100,100,100,100,100,100
186th District Court,100,100,100,100,100,100
187th District Court,85,85,85,85,85,85
222 Main Plaza - Law Office,100,100,100,100,100,100
226th District Court,100,100,100,100,100,100


In [126]:
print('There are {} uniques categories.'.format(len(nearby_val['Venue Category'].unique())))

There are 85 uniques categories.


In [127]:
# one hot encoding
sa_onehot = pd.get_dummies(nearby_val[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sa_onehot['Neighbourhood'] = nearby_val['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [sa_onehot.columns[-1]] + list(sa_onehot.columns[:-1])
sa_onehot = sa_onehot[fixed_columns]

sa_onehot.head()

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Bridge,...,Spanish Restaurant,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Tattoo Parlor,Theater,Theme Restaurant,Thrift / Vintage Store,Wine Bar
0,San Fernando Cathedral,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,San Fernando Cathedral,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,San Fernando Cathedral,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,San Fernando Cathedral,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,San Fernando Cathedral,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
sa_onehot.shape

(18540, 86)

In [148]:
sa_grouped = sa_onehot.groupby('Neighbourhood').mean().reset_index()
sa_grouped

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Bridge,...,Spanish Restaurant,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Tattoo Parlor,Theater,Theme Restaurant,Thrift / Vintage Store,Wine Bar
0,0.0 BEGIN,0.030000,0.010000,0.010000,0.00,0.030000,0.010000,0.070000,0.010000,0.010000,...,0.000000,0.00,0.040000,0.010000,0.010000,0.0,0.040000,0.010000,0.000000,0.010000
1,131st District Court,0.040000,0.010000,0.010000,0.00,0.020000,0.000000,0.040000,0.000000,0.010000,...,0.010000,0.00,0.050000,0.010000,0.010000,0.0,0.040000,0.010000,0.010000,0.010000
2,144th District Court of Bexar County,0.050000,0.010000,0.010000,0.00,0.020000,0.010000,0.040000,0.000000,0.010000,...,0.010000,0.00,0.050000,0.010000,0.010000,0.0,0.030000,0.010000,0.010000,0.010000
3,150th District Court,0.040000,0.010000,0.010000,0.00,0.020000,0.000000,0.030000,0.000000,0.010000,...,0.010000,0.00,0.050000,0.010000,0.010000,0.0,0.040000,0.010000,0.010000,0.010000
4,166th District Court,0.040000,0.010000,0.010000,0.00,0.020000,0.010000,0.030000,0.000000,0.010000,...,0.010000,0.00,0.050000,0.010000,0.010000,0.0,0.040000,0.010000,0.010000,0.010000
5,186th District Court,0.030000,0.010000,0.010000,0.00,0.030000,0.010000,0.040000,0.010000,0.010000,...,0.000000,0.00,0.050000,0.010000,0.010000,0.0,0.030000,0.010000,0.000000,0.010000
6,187th District Court,0.047059,0.000000,0.000000,0.00,0.023529,0.000000,0.011765,0.000000,0.011765,...,0.011765,0.00,0.023529,0.011765,0.023529,0.0,0.035294,0.000000,0.011765,0.011765
7,222 Main Plaza - Law Office,0.040000,0.010000,0.010000,0.00,0.020000,0.010000,0.030000,0.010000,0.010000,...,0.010000,0.00,0.060000,0.010000,0.010000,0.0,0.040000,0.010000,0.010000,0.010000
8,226th District Court,0.040000,0.010000,0.010000,0.00,0.020000,0.010000,0.030000,0.010000,0.010000,...,0.010000,0.00,0.050000,0.010000,0.010000,0.0,0.030000,0.010000,0.010000,0.010000
9,227th District Court,0.040000,0.010000,0.010000,0.00,0.020000,0.010000,0.040000,0.000000,0.010000,...,0.010000,0.00,0.040000,0.010000,0.010000,0.0,0.030000,0.010000,0.010000,0.010000


In [149]:
sa_grouped.shape

(197, 86)

In [150]:
num_top_venues = 5

for hood in sa_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = sa_grouped[sa_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----0.0 BEGIN----
                venue  freq
0               Hotel  0.15
1  Mexican Restaurant  0.07
2                 Bar  0.07
3        Cocktail Bar  0.05
4             Theater  0.04


----131st District Court----
                 venue  freq
0                Hotel  0.18
1   Mexican Restaurant  0.07
2         Cocktail Bar  0.06
3           Steakhouse  0.05
4  American Restaurant  0.04


----144th District Court of Bexar County----
                 venue  freq
0                Hotel  0.14
1   Mexican Restaurant  0.09
2  American Restaurant  0.05
3         Cocktail Bar  0.05
4           Steakhouse  0.05


----150th District Court----
                 venue  freq
0                Hotel  0.18
1   Mexican Restaurant  0.07
2         Cocktail Bar  0.06
3           Steakhouse  0.05
4  American Restaurant  0.04


----166th District Court----
                 venue  freq
0                Hotel  0.17
1   Mexican Restaurant  0.08
2         Cocktail Bar  0.06
3           Steakhouse  0.05
4  Amer

In [151]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [152]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = sa_grouped['Neighbourhood']

for ind in np.arange(sa_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sa_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.0 BEGIN,Hotel,Bar,Mexican Restaurant,Cocktail Bar,Seafood Restaurant,Theater,Steakhouse,Ice Cream Shop,BBQ Joint,Coffee Shop
1,131st District Court,Hotel,Mexican Restaurant,Cocktail Bar,Steakhouse,American Restaurant,Theater,Bar,Sandwich Place,Coffee Shop,Seafood Restaurant
2,144th District Court of Bexar County,Hotel,Mexican Restaurant,American Restaurant,Cocktail Bar,Steakhouse,Seafood Restaurant,Bar,Coffee Shop,Sandwich Place,Ice Cream Shop
3,150th District Court,Hotel,Mexican Restaurant,Cocktail Bar,Steakhouse,American Restaurant,Seafood Restaurant,Theater,Bar,Coffee Shop,Sandwich Place
4,166th District Court,Hotel,Mexican Restaurant,Cocktail Bar,Steakhouse,American Restaurant,Seafood Restaurant,Theater,Bar,Coffee Shop,Sandwich Place
5,186th District Court,Hotel,Mexican Restaurant,Cocktail Bar,Steakhouse,Seafood Restaurant,Bar,Ice Cream Shop,BBQ Joint,Coffee Shop,Sandwich Place
6,187th District Court,Hotel,Mexican Restaurant,American Restaurant,Cocktail Bar,Theater,Coffee Shop,Pizza Place,Sandwich Place,Seafood Restaurant,BBQ Joint
7,222 Main Plaza - Law Office,Hotel,Mexican Restaurant,Cocktail Bar,Steakhouse,American Restaurant,Seafood Restaurant,Theater,Bar,Coffee Shop,Sandwich Place
8,226th District Court,Hotel,Mexican Restaurant,Cocktail Bar,Seafood Restaurant,Steakhouse,American Restaurant,Sandwich Place,Bar,Coffee Shop,Ice Cream Shop
9,227th District Court,Hotel,Mexican Restaurant,Cocktail Bar,American Restaurant,Seafood Restaurant,Bar,Steakhouse,Sandwich Place,Coffee Shop,Pizza Place


In [153]:
from sklearn.cluster import KMeans

In [196]:
# set number of clusters
kclusters = 5

sa_grouped_clustering = sa_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sa_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

In [203]:
sa_grouped.head()

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Bridge,...,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Tattoo Parlor,Theater,Theme Restaurant,Thrift / Vintage Store,Wine Bar,Cluster Labels
0,0.0 BEGIN,0.03,0.01,0.01,0.0,0.03,0.01,0.07,0.01,0.01,...,0.0,0.04,0.01,0.01,0.0,0.04,0.01,0.00,0.01,1
1,131st District Court,0.04,0.01,0.01,0.0,0.02,0.00,0.04,0.00,0.01,...,0.0,0.05,0.01,0.01,0.0,0.04,0.01,0.01,0.01,2
2,144th District Court of Bexar County,0.05,0.01,0.01,0.0,0.02,0.01,0.04,0.00,0.01,...,0.0,0.05,0.01,0.01,0.0,0.03,0.01,0.01,0.01,2
3,150th District Court,0.04,0.01,0.01,0.0,0.02,0.00,0.03,0.00,0.01,...,0.0,0.05,0.01,0.01,0.0,0.04,0.01,0.01,0.01,2
4,166th District Court,0.04,0.01,0.01,0.0,0.02,0.01,0.03,0.00,0.01,...,0.0,0.05,0.01,0.01,0.0,0.04,0.01,0.01,0.01,2


In [198]:
sa_grouped_clustering.shape

(197, 86)

In [192]:
#len(kmeans.labels_)
neighbourhoods_venues_sorted.count()
#sa_merged.head()


Neighbourhood             197
1st Most Common Venue     197
2nd Most Common Venue     197
3rd Most Common Venue     197
4th Most Common Venue     197
5th Most Common Venue     197
6th Most Common Venue     197
7th Most Common Venue     197
8th Most Common Venue     197
9th Most Common Venue     197
10th Most Common Venue    197
dtype: int64

In [188]:
#nearby_val.shape
#dataframe_filtered.shape
sa_grouped.shape

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Bridge,...,Spanish Restaurant,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Tattoo Parlor,Theater,Theme Restaurant,Thrift / Vintage Store,Wine Bar
0,0.0 BEGIN,0.03,0.01,0.01,0.0,0.03,0.01,0.07,0.01,0.01,...,0.00,0.0,0.04,0.01,0.01,0.0,0.04,0.01,0.00,0.01
1,131st District Court,0.04,0.01,0.01,0.0,0.02,0.00,0.04,0.00,0.01,...,0.01,0.0,0.05,0.01,0.01,0.0,0.04,0.01,0.01,0.01
2,144th District Court of Bexar County,0.05,0.01,0.01,0.0,0.02,0.01,0.04,0.00,0.01,...,0.01,0.0,0.05,0.01,0.01,0.0,0.03,0.01,0.01,0.01
3,150th District Court,0.04,0.01,0.01,0.0,0.02,0.00,0.03,0.00,0.01,...,0.01,0.0,0.05,0.01,0.01,0.0,0.04,0.01,0.01,0.01
4,166th District Court,0.04,0.01,0.01,0.0,0.02,0.01,0.03,0.00,0.01,...,0.01,0.0,0.05,0.01,0.01,0.0,0.04,0.01,0.01,0.01


In [211]:
sa_merged = dataframe_filtered

# add clustering labels
sa_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sa_merged = sa_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='name')

sa_merged.head() # check the last columns!

,name,categories,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,San Fernando Cathedral,Church,115 W Main Plz,US,San Antonio,United States,NaN,"[115 W Main Plz, San Antonio, TX 78205, United...","[{'label': 'display', 'lat': 29.42445959712935...",29.424460,...,Hotel,Mexican Restaurant,Cocktail Bar,American Restaurant,Seafood Restaurant,Steakhouse,Sandwich Place,Bar,Coffee Shop,Ice Cream Shop
1,Bexar County Courthouse,Courthouse,100 Dolorosa,US,San Antonio,United States,NaN,"[100 Dolorosa, San Antonio, TX 78205, United S...","[{'label': 'display', 'lat': 29.42321234496511...",29.423212,...,Hotel,Mexican Restaurant,Cocktail Bar,American Restaurant,Bar,Seafood Restaurant,BBQ Joint,Theater,Steakhouse,Coffee Shop
2,227th District Court,Courthouse,300 Dolorosa,US,San Antonio,United States,Dwyer,"[300 Dolorosa (Dwyer), San Antonio, TX 78205, ...","[{'label': 'display', 'lat': 29.4243144745312,...",29.424314,...,Hotel,Mexican Restaurant,Cocktail Bar,American Restaurant,Seafood Restaurant,Bar,Steakhouse,Sandwich Place,Coffee Shop,Pizza Place
3,Main Plaza Park,Plaza,100 W Main Plz,US,San Antonio,United States,btwn E Commerce St & W Market/Dolorosa,[100 W Main Plz (btwn E Commerce St & W Market...,"[{'label': 'display', 'lat': 29.42458767477948...",29.424588,...,Hotel,Mexican Restaurant,Cocktail Bar,Steakhouse,Seafood Restaurant,Bar,Ice Cream Shop,BBQ Joint,Coffee Shop,Sandwich Place
4,Main Plaza Farmer's Market,General Entertainment,NaN,US,San Antonio,United States,NaN,"[San Antonio, TX 78205, United States]","[{'label': 'display', 'lat': 29.42475970408229...",29.424760,...,Hotel,Mexican Restaurant,American Restaurant,Seafood Restaurant,Cocktail Bar,Steakhouse,Bar,Sandwich Place,Coffee Shop,Ice Cream Shop


In [216]:
sa_merged.loc[sa_merged['Cluster Labels'] == 0, sa_merged.columns[[1] + list(range(5, sa_merged.shape[1]))]]

,categories,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Courthouse,United States,NaN,"[100 Dolorosa, San Antonio, TX 78205, United S...","[{'label': 'display', 'lat': 29.42321234496511...",29.423212,-98.493719,78205,TX,4bb08897f964a520464a3ce3,...,Hotel,Mexican Restaurant,Cocktail Bar,American Restaurant,Bar,Seafood Restaurant,BBQ Joint,Theater,Steakhouse,Coffee Shop
2,Courthouse,United States,Dwyer,"[300 Dolorosa (Dwyer), San Antonio, TX 78205, ...","[{'label': 'display', 'lat': 29.4243144745312,...",29.424314,-98.493832,78205,TX,4c814f67dc018cfabf7ec46c,...,Hotel,Mexican Restaurant,Cocktail Bar,American Restaurant,Seafood Restaurant,Bar,Steakhouse,Sandwich Place,Coffee Shop,Pizza Place
3,Plaza,United States,btwn E Commerce St & W Market/Dolorosa,[100 W Main Plz (btwn E Commerce St & W Market...,"[{'label': 'display', 'lat': 29.42458767477948...",29.424588,-98.493285,78205,TX,4bbe1906b42aef3b97664bc3,...,Hotel,Mexican Restaurant,Cocktail Bar,Steakhouse,Seafood Restaurant,Bar,Ice Cream Shop,BBQ Joint,Coffee Shop,Sandwich Place
4,General Entertainment,United States,NaN,"[San Antonio, TX 78205, United States]","[{'label': 'display', 'lat': 29.42475970408229...",29.424760,-98.493585,78205,TX,50b4feace4b0818e9afd0b96,...,Hotel,Mexican Restaurant,American Restaurant,Seafood Restaurant,Cocktail Bar,Steakhouse,Bar,Sandwich Place,Coffee Shop,Ice Cream Shop
5,Historic Site,United States,NaN,"[300 Dolorosa, San Antonio, TX 78205, United S...","[{'label': 'display', 'lat': 29.42394637933925...",29.423946,-98.493473,78205,TX,4e1b2a241495517ce518b66b,...,Hotel,Mexican Restaurant,Cocktail Bar,Seafood Restaurant,Steakhouse,American Restaurant,Bar,Sandwich Place,Coffee Shop,Ice Cream Shop
7,Wine Bar,United States,at Navarro St,"[214 W Crockett St (at Navarro St), San Antoni...","[{'label': 'display', 'lat': 29.42496387179004...",29.424964,-98.490272,78205,TX,4b8556eef964a520ab5731e3,...,Hotel,Bar,Steakhouse,Mexican Restaurant,Ice Cream Shop,Seafood Restaurant,Theater,Cocktail Bar,Historic Site,New American Restaurant
8,Theater,United States,NaN,"[208 E Houston St, San Antonio, TX 78205, Unit...","[{'label': 'display', 'lat': 29.42598141664784...",29.425981,-98.491150,78205,TX,4c40ccbbe26920a18d795de7,...,Hotel,Bar,Mexican Restaurant,Seafood Restaurant,Theater,Cocktail Bar,Steakhouse,Ice Cream Shop,Coffee Shop,Historic Site
9,Food Truck,United States,NaN,"[San Antonio, TX, United States]","[{'label': 'display', 'lat': 29.424185, 'lng':...",29.424185,-98.493505,NaN,TX,54320e11498eec43385d369b,...,Hotel,Mexican Restaurant,Cocktail Bar,Seafood Restaurant,Steakhouse,American Restaurant,Sandwich Place,Bar,Coffee Shop,Ice Cream Shop
12,Courthouse,United States,NaN,"[San Antonio, TX, United States]","[{'label': 'display', 'lat': 29.42414414786921...",29.424144,-98.493591,NaN,TX,4e26f07fa809510950dbdc92,...,Hotel,Mexican Restaurant,Cocktail Bar,Seafood Restaurant,Steakhouse,American Restaurant,Bar,Sandwich Place,Coffee Shop,Ice Cream Shop
15,Beer Garden,United States,NaN,"[San Antonio, TX 78205, United States]","[{'label': 'display', 'lat': 29.42412130408513...",29.424121,-98.493628,78205,TX,5027305fe4b0b57d7202a1a2,...,Hotel,Mexican Restaurant,Cocktail Bar,Steakhouse,American Restaurant,Seafood Restaurant,Bar,Sandwich Place,Coffee Shop,Pizza Place
